# ODEbase models

In [1]:
import sys

sys.path.insert(0,"./../tests/numerical") # clue_example is here
sys.path.insert(0,"./../") # clue is here
from clue import *
from numerical_example import *
# import numerical_example as ne
from sympy import RR
import pandas as pd
import time
from alive_progress import alive_bar
from IPython.display import Image, display
import logging
logger = logging.getLogger("clue")
from clue.odebase_io import *
from clue.ode_parser import *


2023-11-23 13:30:01 ERROR    [Load_Examples_Folder] Example epsCLump_ei is not a valid example. Make sure it has a valid ref or all three read, matrix, and observables fields
2023-11-23 13:30:01 ERROR    [Load_Examples_Folder] Example epsclump is not a valid example. Make sure it has a valid ref or all three read, matrix, and observables fields
2023-11-23 13:30:01 ERROR    [Load_Examples_Folder] Example epsclumpBNBScaff22 is not a valid example. Make sure it has a valid ref or all three read, matrix, and observables fields
2023-11-23 13:30:01 ERROR    [Load_Examples_Folder] Example epsclumpBNG is not a valid example. Make sure it has a valid ref or all three read, matrix, and observables fields
2023-11-23 13:30:01 ERROR    [Load_Examples_Folder] Example epsclumpBNG2 is not a valid example. Make sure it has a valid ref or all three read, matrix, and observables fields
2023-11-23 13:30:01 ERROR    [Load_Examples_Folder] Example epsclumpFCeri is not a valid example. Make sure it has a val

In [2]:
get_odes()

TypeError: get_odes() missing 2 required positional arguments: 'base_url' and 'ref'

In [6]:
def ode_scrapper(*, 
                 polynomial: bool=None, rational:bool=None, 
                 num_species: int|tuple[int,int]=None, 
                 num_parameters: int|tuple[int,int]=None, 
                 num_constraints: int|tuple[int,int]=None,
                 deficiency: int|tuple[int,int]=None,
                 id: int = None,
                 name: str = None,
                 index: int = None,
                 translation: bool = False) -> FODESystem | list[FODESystem]:
    r'''
        Method to create the :class:`FODESystem` from an entry in the ODEBase Database.

        All arguments are named, meaning we need to provide them as keywords. They provide an 
        interface for filtering in the database. We return a list whenever there is more than one 
        system that satisfy all the criteria.

        The arguments can be as follows:

        * ``polynomial``: boolean value indicating whether the system is polynomial or not.
        * ``rational``: boolean value indicating whether the system is rational or not. Be aware that 
          any polynomial system is also rational, hence setting ``rational`` to ``False`` will also
          exclude all polynomial models.
        * ``num_species``: either a number or a pair of numbers indicating the range of the 
          number of species allowed in the system. The format ``(None, d)`` will return all models
          with number of species up to `d`, while ``(d, None)`` return the models with at least `d`
          species. A single value indicates an exact number of species derired.
        * ``num_parameters``: same as ``num_species`` but with the number of parameters of the system.
        * ``num_constrains``: same as ``num_species`` but with the number of constrains of the system.
        * ``deficiency``: same as ``num_species`` but with the number of deficiencies of the system.
        * ``id``: obtains the model with given ID if it has passed the other filters
        * ``index``: if obtained several models after filtering, it returns the given in that index.
        * ``translation``: models in ODEBase are usually defined with predefined names that are then 
          map to the real values of their corresponding meaning. This flag indicates if the obtained
          model must translate the names to the true names or not.
    '''
    # URL of the website to analyze
    base_url = "https://www.odebase.org"
    rational_url = f"is_rational={rational}" if rational != None else ""
    polynomial_url = f"is_polynomial={polynomial}" if polynomial != None else ""
    if isinstance(num_species, int):
        num_species = f"{num_species}"
    elif isinstance(num_species, tuple):
        num_species = f"{f'{num_species[0]}' if num_species[0] != None else ''}-{f'{num_species[1]}' if num_species[1] != None else ''}"
    if isinstance(num_parameters, int):
        num_parameters = f"{num_parameters}"
    elif isinstance(num_parameters, tuple):
        num_parameters = f"{f'{num_parameters[0]}' if num_parameters[0] != None else ''}-{f'{num_parameters[1]}' if num_parameters[1] != None else ''}"
    if isinstance(num_constraints, int):
        num_constraints = f"{num_constraints}"
    elif isinstance(num_constraints, tuple):
        num_constraints = f"{f'{num_constraints[0]}' if num_constraints[0] != None else ''}-{f'{num_constraints[1]}' if num_constraints[1] != None else ''}"
    if isinstance(deficiency, int):
        deficiency = f"{deficiency}"
    elif isinstance(deficiency, tuple):
        deficiency = f"{f'{deficiency[0]}' if deficiency[0] != None else ''}-{f'{deficiency[1]}' if deficiency[1] != None else ''}"
    num_species_url = f"num_species_range={num_species}" if num_species != None else ""
    num_parameters_url = f"num_parameters_range={num_parameters}" if num_parameters != None else ""
    num_constraints_url = f"num_constraints_range={num_constraints}" if num_constraints != None else ""
    deficiency_url = f"deficiency_range={deficiency}" if deficiency != None else ""
    url = f"{base_url}/table/?{'&'.join(el for el in [rational_url, polynomial_url, num_species_url, num_parameters_url, num_constraints_url, deficiency_url] if el != '')}"
    
    # Send an HTTP GET request to the website
    logger.debug("[scrap] Getting the list of Rational models...")
    response = requests.get(url)

    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(io.BytesIO(response.content), 'html.parser')

    # Extract the relevant information from the HTML code
    references = list()
    for row in soup.select('tbody tr'):
        all_tds = row.find_all("td")
        model_ref = all_tds[1].find("a")
        mod_name = model_ref.get_text()
        ref = int(model_ref.get("href").split("/")[-1]) # just the number
        is_polynomial = all_tds[8].get_text() == "Yes"
        references.append((mod_name, ref, is_polynomial))

    if index != None: # we only keep the indexed reference
        references = [references[index]]
    if id != None: # we only keep the given id
        references = [(mod_name, ref, is_poly) for (mod_name,ref,is_poly) in references if ref == id]
    if name != None: # we only keep the given name
        references = [(mod_name, ref, is_poly) for (mod_name,ref, is_poly) in references if mod_name == name]
    
        
    logger.debug(f"[scrap] Obtained {len(references)} models")
    logger.debug(f"[scrap] Processing each model...")
    output = list()
       
    for i,(name, ref, is_poly) in enumerate(references):
        logger.debug(f"[scrap] Processing model {name} ({i+1}/{len(references)})...")
        
        try:
            output.append(get_clue(base_url, ref, name=name, transform_names=translation, polynomial=is_poly))
        except (TypeError,ValueError): # We first try to simplify the reading
            try:
                output.append(get_clue(base_url, ref, name=name, transform_names=translation, polynomial=False))
            except (TypeError,ValueError):
                try:
                    output.append(get_clue(base_url, ref, name=name, transform_names=False, polynomial=is_poly))
                except (TypeError, ValueError):
                    try:
                        output.append(get_clue(base_url, ref, name=name, transform_names=False, polynomial=False))
                    except (TypeError, ValueError):
                        logger.error(f"[scrap] Found error while parsing model {name}.")
                        raise        
    logger.debug(f"[scrap] Finished execution")
    return output if len(output)!=1 else output[0]

In [111]:
s = ode_scrapper(polynomial=True, index=0,num_species=(5,None))
name = s.name.split()[2][1:-1]
lines,t_horizon = get_ic_for_states(name)
print(s.ic)
ic = parse_initial_conditions(lines, prev_ic=s.ic)
s.ic.update(ic)
print(ic)
s.ic


{'k1': mpq(300000000,1), 'k2': mpq(8000,1), 'k3': mpq(150000000,1), 'k4': mpq(16000,1), 'k5': mpq(30000,1), 'k6': mpq(700,1), 'k7': mpq(300000000,1), 'k8': mpq(216,25), 'k9': mpq(150000000,1), 'k10': mpq(432,25), 'k11': mpq(27,50), 'k12': mpq(10800,1), 'k13': mpq(130,1), 'k14': mpq(2740,1), 'k15': mpq(300000000,1), 'k16': mpq(4,1), 'k17': mpq(150000000,1), 'k18': mpq(8,1), 'k19': mpq(197,10), 'k20': mpq(187,50), 'k21': mpq(397,20), 'k22': mpq(87,50), 'k23': mpq(20,1), 'k24': mpq(81,100), 'k25': mpq(300000000,1), 'k26': mpq(4,1), 'k27': mpq(150000000,1), 'k28': mpq(8,1), 'k29': mpq(1,20), 'k30': mpq(3,2500), 'k31': mpq(1,20), 'k32': mpq(3,2500), 'k33': mpq(1,20), 'k34': mpq(3,2500), 'k35': mpq(1,10000000000000000), 'k36': mpq(1,10000000000000000000000), 'k37': mpq(1,1000000000000000000000)}
{'x1': mpq(0,1), 'x2': mpq(0,1), 'x3': mpq(0,1), 'x4': mpq(0,1), 'x5': mpq(0,1), 'x6': mpq(1,1), 'x7': mpq(0,1), 'x8': mpq(0,1), 'x9': mpq(0,1), 'x10': mpq(0,1), 'x11': mpq(0,1), 'x12': mpq(0,1), 'x1

{'k1': mpq(300000000,1),
 'k2': mpq(8000,1),
 'k3': mpq(150000000,1),
 'k4': mpq(16000,1),
 'k5': mpq(30000,1),
 'k6': mpq(700,1),
 'k7': mpq(300000000,1),
 'k8': mpq(216,25),
 'k9': mpq(150000000,1),
 'k10': mpq(432,25),
 'k11': mpq(27,50),
 'k12': mpq(10800,1),
 'k13': mpq(130,1),
 'k14': mpq(2740,1),
 'k15': mpq(300000000,1),
 'k16': mpq(4,1),
 'k17': mpq(150000000,1),
 'k18': mpq(8,1),
 'k19': mpq(197,10),
 'k20': mpq(187,50),
 'k21': mpq(397,20),
 'k22': mpq(87,50),
 'k23': mpq(20,1),
 'k24': mpq(81,100),
 'k25': mpq(300000000,1),
 'k26': mpq(4,1),
 'k27': mpq(150000000,1),
 'k28': mpq(8,1),
 'k29': mpq(1,20),
 'k30': mpq(3,2500),
 'k31': mpq(1,20),
 'k32': mpq(3,2500),
 'k33': mpq(1,20),
 'k34': mpq(3,2500),
 'k35': mpq(1,10000000000000000),
 'k36': mpq(1,10000000000000000000000),
 'k37': mpq(1,1000000000000000000000),
 'x1': mpq(0,1),
 'x2': mpq(0,1),
 'x3': mpq(0,1),
 'x4': mpq(0,1),
 'x5': mpq(0,1),
 'x6': mpq(1,1),
 'x7': mpq(0,1),
 'x8': mpq(0,1),
 'x9': mpq(0,1),
 'x10': mp

In [6]:
model = "BIOMD0000000199" # troublesome
def get_ic_for_states(model):
    api_url = "https://www.ebi.ac.uk/biomodels/"
    url = api_url+model+"#Files"
    response = requests.get(api_url + model + "?format=json")
    sub_id = response.json()["submissionId"]
    url = api_url+"services/download/get-files/"+sub_id+"/2/"+model+".m"
    response = requests.get(url)
    m_file = response.text

    init_cond = re.findall(r"x0\((\d*)\) = (\d*.\d*);", m_file)
    dic_ic = {}
    ic_lines = []
    for cond in init_cond:
        dic_ic["x"+cond[0]] = cond[1]
        ic_lines.append("x" +cond[0]+" = "+cond[1])

    t_horizon = re.findall("t=linspace\(0,(\d*),\d*\)", m_file)[0]
    
    return ic_lines, t_horizon
ic_lines,_= get_ic_for_states(model)
# print(dic_ic)
print(ic_lines)
# print(t_horizon)        
# print(m_file)
# simulateODE(tEnd=100,viewPlot=VIEWS)





['x1 = 1.0', 'x2 = 0.0', 'x3 = 0.0', 'x4 = 0.0', 'x5 = 0.0', 'x6 = 0.0', 'x7 = 0.0', 'x8 = 0.0', 'x9 = 40.0', 'x10 = 140.0', 'x11 = 0.0', 'x12 = 0.0', 'x13 = 0.0', 'x14 = 0.0', 'x15 = 0']


In [7]:
import re
r = "(BIOMD\d*)"
path = "./odebase/"
base_url = "https://www.odebase.org"
for indx in range(90):
    s = ode_scrapper(polynomial=True, index=indx,num_species=(5,None))
    if type(s.equations[0]) is SparsePolynomial:
        ref = s.name.split()[1]
        variables = get_dictionary_of_variables(base_url, 'var', ref )
        name = s.name.split()[2][1:-1]
        lines,t_horizon = get_ic_for_states(name)
        ic = parse_initial_conditions(lines, prev_ic=s.ic)
        s.ic.update(ic)
        file=path+name+".ode"
        s.save(path+name,format="ode")
        with open(file, "a") as f:
            f.write("//simulateODE(tEnd="+t_horizon+",viewPlot=VIEWS)\n") 
            f.write("//Species\n")
            for var in variables:
                f.write("//"+var+":"+variables[var]+"\n")


# Models Andrea

In [2]:
import sys

sys.path.insert(0,"./../tests/numerical") # clue_example is here
sys.path.insert(0,"./../") # clue is here
from clue import *
from numerical_example import *

In [3]:
path = '/home/alexander/CLUE/models/rational/'
add_examples_in_folder("-ps", "-r", "rational","-m","rational", "-ow","-of", "./rational-sizes", "-j", "./rational-sizes.json", path)

2023-12-01 17:03:16 INFO     [get_model_dict] The file /home/alexander/CLUE/models/rational/MODEL1502270000.ode does not have any explicit views.
2023-12-01 17:03:16 INFO     [get_model_dict] The file /home/alexander/CLUE/models/rational/MODEL1502270000.ode does not have partitions to generate view from.
2023-12-01 17:03:16 WARNING  [get_model_dict] File /home/alexander/CLUE/models/rational/MODEL1502270000.ode does not have observables.
2023-12-01 17:03:16 INFO     [write_experiments_to_json] Model BIOMD0000000013 already found.
2023-12-01 17:03:16 INFO     [write_experiments_to_json] Overwriting model BIOMD0000000013
2023-12-01 17:03:16 INFO     [write_experiments_to_json] Model BIOMD0000000023 already found.
2023-12-01 17:03:16 INFO     [write_experiments_to_json] Overwriting model BIOMD0000000023
2023-12-01 17:03:16 INFO     [write_experiments_to_json] Model BIOMD0000000033 already found.
2023-12-01 17:03:16 INFO     [write_experiments_to_json] Overwriting model BIOMD0000000033
2023

# New found models

In [2]:
import sys

sys.path.insert(0,"./../tests/numerical") # clue_example is here
sys.path.insert(0,"./../") # clue is here

from clue import *
from numerical_example import *
from sympy import RR
import pandas as pd
import time
from alive_progress import alive_bar
from IPython.display import Image, display